# FQL Utilities Demonstration

This notebook demonstrates the FQL (Fiddler Query Language) utility functions available in the `fiddler_utils` package. These utilities help you parse, validate, transform, and analyze FQL expressions used in segments, custom metrics, and other Fiddler assets.

## Prerequisites

* Access to a Fiddler environment with at least one model containing [segments](https://docs.fiddler.ai/product-guide/monitoring-platform/segments) or [custom metrics](https://docs.fiddler.ai/product-guide/monitoring-platform/custom-metrics)
* [API toke](https://docs.fiddler.ai/configuration-guide/settings#credentials)n with [read access](https://docs.fiddler.ai/configuration-guide/access-control/role-based-access#understanding-permissions) (write access needed for Section 5 examples)
* Python packages: [fiddler-client](https://docs.fiddler.ai/technical-reference/python-client-guides/installation-and-setup), `fiddler_utils`

## What is FQL?

[FQL is Fiddler's query language](https://docs.fiddler.ai/product-guide/monitoring-platform/fiddler-query-language) for defining:
* **Segments** - Filter expressions to define data subsets
* **Custom Metrics** - Calculated metrics using aggregation functions
* **Alert Rules** - Conditions for triggering alerts

### FQL Syntax Rules

* **Column names:** Always in double quotes (e.g., `"column_name"`)
* **String values:** Always in single quotes (e.g., `'value'`)
* **Numeric values:** No quotes (e.g., `42`, `3.14`)
* **Operators:** `==`, `!=`, `>`, `<`, `>=`, `<=`, `and`, `or`, `not`
* **Functions:** `sum()`, `avg()`, `if()`, `fp()`, `fn()`, `tp()`, `tn()`, etc.

### Configuration

## Table of Contents

**Quick Navigation:**

1. [**Section 1: Standalone FQL Utilities**](#Section-1:-Standalone-FQL-Utilities)
   * [1.1 Extract Column References](#1.1-Extract-Column-References)
   * [1.2 Validate FQL Syntax](#1.2-Validate-FQL-Syntax)
   * [1.3 Normalize Expressions](#1.3-Normalize-Expressions)
   * [1.4 Extract FQL Functions](#1.4-Extract-FQL-Functions)
   * [1.5 Distinguish Simple Filters from Aggregations](#1.5-Distinguish-Simple-Filters-from-Aggregations)
   * [1.6 Split AND Conditions](#1.6-Split-AND-Conditions)

2. [**Section 2: Live Fiddler Integration**](#Section-2:-Live-Fiddler-Integration)
   * [2.1 Connect to Fiddler](#2.1-Connect-to-Fiddler)
   * [2.2 List Available Projects and Models](#2.2-List-Available-Projects-and-Models)
   * [2.3 Get Model and Analyze Segments](#2.3-Get-Model-and-Analyze-Segments)
   * [2.4 Analyze Custom Metrics](#2.4-Analyze-Custom-Metrics)

3. [**Section 3: Column Mapping and Asset Migration**](#Section-3:-Column-Mapping-and-Asset-Migration)
   * [3.1 Simple Column Name Replacement](#3.1-Simple-Column-Name-Replacement)
   * [3.2 Complex Expression Transformation](#3.2-Complex-Expression-Transformation)
   * [3.3 Interactive Column Mapping Builder](#3.3-Interactive-Column-Mapping-Builder)
   * [3.4 End-to-End Migration Example](#3.4-End-to-End-Migration-Example)
   * [3.5 Batch Migration with Validation](#3.5-Batch-Migration-with-Validation)

4. [**Section 4: Advanced Patterns and Best Practices**](#Section-4:-Advanced-Patterns-and-Best-Practices)
   * [4.1 Comprehensive Expression Validation Pipeline](#4.1-Comprehensive-Expression-Validation-Pipeline)
   * [4.2 Expression Comparison and Deduplication](#4.2-Expression-Comparison-and-Deduplication)
   * [4.3 Safe Expression Modification Workflow](#4.3-Safe-Expression-Modification-Workflow)
   * [4.4 Expression Analysis Report](#4.4-Expression-Analysis-Report)

5. [**Section 5: UUID Reference Management & Safe Metric Updates**](#Section-5:-UUID-Reference-Management-&-Safe-Metric-Updates) 🆕
   * [5.1 Import Reference Management Utilities](#5.1-Import-Reference-Management-Utilities)
   * [5.2 Demonstrating the UUID Problem](#5.2-Demonstrating-the-UUID-Problem)
   * [5.3 Finding All References Before Updating](#5.3-Finding-All-References-Before-Updating)
   * [5.4 Safe Metric Update with Automatic Reference Migration](#5.4-Safe-Metric-Update-with-Automatic-Reference-Migration)

6. [**Section 6: Testing FQL Before Creating Metrics**](#Section-6:-Testing-FQL-Before-Creating-Metrics) 🆕
   * [6.1 Import Testing Utilities](#6.1-Import-Testing-Utilities)
   * [6.2 Local Pre-Validation (Fast)](#6.2-Local-Pre-Validation-(Fast))
   * [6.3 Real Testing with Temporary Metrics](#6.3-Real-Testing-with-Temporary-Metrics)
   * [6.4 Complete Validation Workflow](#6.4-Complete-Validation-Workflow)
   * [6.5 Batch Testing Multiple Metrics](#6.5-Batch-Testing-Multiple-Metrics)
   * [6.6 Cleanup Orphaned Test Metrics](#6.6-Cleanup-Orphaned-Test-Metrics)

---

### Import Required Libraries

In [ ]:
import sys
from typing import Dict, Set

import fiddler as fdl

from fiddler_utils import fql
from fiddler_utils.connection import get_or_init

# Add parent directory to path to import fiddler_utils
sys.path.insert(0, "..")

print("✓ Imports successful")


In [ ]:
# Fiddler environment configuration
URL = ""  # Example: 'https://your_company_name.fiddler.ai'
TOKEN = ""  # Your API token

# Model to use for examples (we'll list available models if not specified)
PROJECT_NAME = ""  # Example: 'my_project'
MODEL_NAME = ""  # Example: 'my_model'
MODEL_VERSION = ""  # Example: 'v1' (optional)

# Set to False to actually execute modifications in Section 5
DRY_RUN = True

---

## Section 1: Standalone FQL Utilities

These functions work with FQL expressions directly, without requiring a Fiddler connection.

### 1.1 Extract Column References

The `extract_columns()` function identifies all column names referenced in an FQL expression.

In [ ]:
# Example FQL expressions
examples = [
    '"age" > 30 and "geography" == \'California\'',
    'sum(if(fp(), 1, 0) * "transaction_value")',
    '"credit_score" >= 700 and "loan_amount" < 50000 and "region" == \'West\'',
    'avg("response_time") > 100',
    '(sum(if(("probability_churn">0.8 and "gender"== \'Nonbinary\'), 1, 0))/sum(if(("gender"== \'Nonbinary\'), 1, 0)))/(sum(if(("probability_churn">0.8 and "gender"== \'Male\'), 1, 0))/sum(if(("gender"== \'Male\'), 1, 0)))',
]

print("Column Extraction Examples:\n")
for expr in examples:
    columns = fql.extract_columns(expr)
    print(f"Expression: {expr}")
    print(f"  Columns: {columns}")
    print(f"  Count: {len(columns)}\n")

### 1.2 Validate FQL Syntax

The `validate_fql_syntax()` function performs basic syntax validation to catch common errors.

In [ ]:
# Valid and invalid FQL expressions
test_expressions = [
    ('"age" > 30', "Valid simple expression"),
    ('"unclosed > 30', "Unbalanced double quotes"),
    ('"age" > \'30', "Unbalanced single quotes"),
    ("sum(if(fp(), 1, 0)", "Unbalanced parentheses"),
    ("\"\" == 'value'", "Empty column reference"),
    ('"status" == \'active\' and "verified" == true', "Valid complex expression"),
]

print("FQL Syntax Validation:\n")
for expr, description in test_expressions:
    is_valid, error_msg = fql.validate_fql_syntax(expr)
    status = "✓" if is_valid else "✗"
    print(f"{status} {description}")
    print(f"  Expression: {expr}")
    if not is_valid:
        print(f"  Error: {error_msg}")
    print()

### 1.3 Normalize Expressions

The `normalize_expression()` function standardizes whitespace and formatting for comparison.

In [ ]:
# Expressions with inconsistent formatting
messy_expressions = [
    '"age"   >  30',
    '"status"=="active"',
    "sum(  if(  fp(  ),1,0)  )",
    "\"region\"   in  ['West','East','North']",
]

print("Expression Normalization:\n")
for expr in messy_expressions:
    normalized = fql.normalize_expression(expr)
    print(f"Original:    {expr}")
    print(f"Normalized:  {normalized}\n")

### 1.4 Extract FQL Functions

The `get_fql_functions()` function identifies all function calls in an expression.

In [ ]:
# Expressions with various functions
function_examples = [
    ("sum(if(fp(), 1, 0))", "Custom metric with false positives"),
    ('avg("response_time")', "Simple average"),
    ("count(if(\"status\" == 'failed', 1, 0))", "Conditional count"),
    ('sum(if(tp(), "revenue", 0)) - sum(if(fp(), "cost", 0))', "Net value calculation"),
    (
        '(sum(if(("probability_churn">0.8 and "gender"== \'Nonbinary\'), 1, 0))/sum(if(("gender"== \'Nonbinary\'), 1, 0)))/(sum(if(("probability_churn">0.8 and "gender"== \'Male\'), 1, 0))/sum(if(("gender"== \'Male\'), 1, 0)))',
        "Disparate Impact Non Binary",
    ),
]

print("FQL Function Extraction:\n")
for expr, description in function_examples:
    functions = fql.get_fql_functions(expr)
    print(f"Description: {description}")
    print(f"  Expression: {expr}")
    print(f"  Functions: {functions}\n")

### 1.5 Distinguish Simple Filters from Aggregations

The `is_simple_filter()` function helps determine if an expression is a simple filter (usable in segments) or contains aggregations (typically for custom metrics).

In [ ]:
# Mix of simple and complex expressions
classification_examples = [
    ('"age" > 30 and "status" == \'active\'', "Segment filter"),
    ("sum(if(fp(), 1, 0))", "Custom metric"),
    ("\"region\" in ['West', 'East']", "Segment filter with list"),
    ('avg("transaction_value")', "Aggregation metric"),
    ('if("premium" == true, "discount", 0)', "Conditional (no aggregation)"),
]

print("Expression Classification:\n")
for expr, description in classification_examples:
    is_simple = fql.is_simple_filter(expr)
    expr_type = "Simple Filter" if is_simple else "Aggregation/Complex"
    icon = "📊" if is_simple else "📈"
    print(f"{icon} {description}")
    print(f"  Type: {expr_type}")
    print(f"  Expression: {expr}\n")

### 1.6 Split AND Conditions

The `split_fql_and_condition()` function breaks down complex filter expressions into individual conditions.

In [ ]:
# Complex expressions with AND conditions
complex_expr = '"age" > 30 and "geography" == \'California\' and "credit_score" >= 700'

print("Splitting AND Conditions:\n")
print("Original expression:")
print(f"  {complex_expr}\n")

parts = fql.split_fql_and_condition(complex_expr)
print(f"Split into {len(parts)} conditions:\n")
for i, part in enumerate(parts, 1):
    print(f"  {i}. {part}")

# Note: This is a simple split and may not handle all cases
print(
    "\n⚠️ Note: Simple implementation - may not handle 'and' inside function calls correctly"
)

---

## Section 2: Live Fiddler Integration

Connect to your Fiddler environment and analyze real FQL expressions from existing assets.

### 2.1 Connect to Fiddler

In [ ]:
# Initialize Fiddler client
if URL and TOKEN:
    get_or_init(url=URL, token=TOKEN, log_level="ERROR")
    print("✓ Connected to Fiddler")
else:
    print("⚠️ Please set URL and TOKEN in the configuration section above")

### 2.2 List Available Projects and Models

In [ ]:
# List all projects
if URL and TOKEN:
    projects = list(fdl.Project.list())
    print(f"Available Projects ({len(projects)}):\n")

    for project in projects[:10]:  # Show first 10
        try:
            models = list(fdl.Model.list(project_id=project.id))
            print(f"📁 {project.name}")
            for model in models[:5]:  # Show first 5 models per project
                print(f"  └─ {model.name} (ID: {model.id})")
            if len(models) > 5:
                print(f"  └─ ... and {len(models) - 5} more models")
        except Exception as e:
            print(f"  └─ Error listing models: {e}")
        print()

    if len(projects) > 10:
        print(f"... and {len(projects) - 10} more projects")

    print("\nℹ️ Set PROJECT_NAME and MODEL_NAME above to focus on a specific model")

### 2.3 Get Model and Analyze Segments

In [ ]:
# Get specified model or use first available
if URL and TOKEN:
    if PROJECT_NAME and MODEL_NAME:
        try:
            project = fdl.Project.get_or_create(name=PROJECT_NAME)
            model = fdl.Model.from_name(
                name=MODEL_NAME, project_id=project.id, version=MODEL_VERSION
            )
            print(f"✓ Using model: {project.name}/{model.name}/{model.version}")
        except fdl.NotFound:
            print(f"✗ Model not found: {PROJECT_NAME}/{MODEL_NAME}/{MODEL_VERSION}")
            print("  Using first available model instead...")
            model = None
        except Exception as e:
            print(f"✗ Error getting model: {e}")
            print("  Using first available model instead...")
            model = None
    else:
        model = None

    # Fallback to first model with segments
    if model is None:
        for project in fdl.Project.list():
            try:
                models = list(fdl.Model.list(project_id=project.id))
                for m in models:
                    segments = list(fdl.Segment.list(model_id=m.id))
                    if segments:
                        model = m
                        print(
                            f"✓ Using model: {project.name}/{model.name} (found {len(segments)} segments)"
                        )
                        break
                if model:
                    break
            except Exception as e:
                print(f"  An error occurred: {e}")
                continue

    if model is None:
        print("⚠️ No models with segments found. Skipping live examples.")

In [ ]:
# Analyze segments from the model
if URL and TOKEN and model:
    try:
        segments = list(fdl.Segment.list(model_id=model.id))

        if segments:
            print(f"Analyzing {len(segments)} Segments:\n")

            for segment in segments:
                print(f"📊 Segment: {segment.name}")
                print(f"  Expression: {segment.definition}")

                # Extract columns
                columns = fql.extract_columns(segment.definition)
                print(f"  Columns: {columns}")

                # Check if simple filter
                is_simple = fql.is_simple_filter(segment.definition)
                print(
                    f"  Type: {'Simple filter' if is_simple else 'Contains aggregations'}"
                )

                # Validate syntax
                is_valid, error = fql.validate_fql_syntax(segment.definition)
                status = "✓ Valid" if is_valid else f"✗ Invalid: {error}"
                print(f"  Syntax: {status}")

                # Get functions used
                functions = fql.get_fql_functions(segment.definition)
                if functions:
                    print(f"  Functions: {functions}")

                print()
        else:
            print("ℹ️ No segments found for this model")
    except Exception as e:
        print(f"✗ Error analyzing segments: {e}")

### 2.4 Analyze Custom Metrics

In [ ]:
# Analyze custom metrics from the model
if URL and TOKEN and model:
    try:
        custom_metrics = list(fdl.CustomMetric.list(model_id=model.id))

        if custom_metrics:
            print(f"Analyzing {len(custom_metrics)} Custom Metrics:\n")

            for metric in custom_metrics:
                print(f"📈 Custom Metric: {metric.name}")
                print(f"  Definition: {metric.definition}")

                # Extract columns
                columns = fql.extract_columns(metric.definition)
                print(f"  Columns: {columns}")

                # Get functions used
                functions = fql.get_fql_functions(metric.definition)
                print(f"  Functions: {functions}")

                # Check if it's actually a simple filter (unusual for custom metrics)
                is_simple = fql.is_simple_filter(metric.definition)
                if is_simple:
                    print("  ⚠️ No aggregations detected (unusual for custom metrics)")

                # Validate syntax
                is_valid, error = fql.validate_fql_syntax(metric.definition)
                status = "✓ Valid" if is_valid else f"✗ Invalid: {error}"
                print(f"  Syntax: {status}")

                print()
        else:
            print("ℹ️ No custom metrics found for this model")
    except Exception as e:
        print(f"✗ Error analyzing custom metrics: {e}")

---

## Section 3: Column Mapping and Asset Migration

Transform FQL expressions when migrating assets between models with different schemas.

### 3.1 Simple Column Name Replacement

In [ ]:
# Example: Renaming columns in an expression
original_expr = '"age" > 30 and "geography" == \'California\' and "credit_score" >= 700'

# Define column mapping (old_name -> new_name)
column_mapping = {
    "age": "customer_age",
    "geography": "location",
    "credit_score": "fico_score",
}

print("Simple Column Name Replacement:\n")
print("Original expression:")
print(f"  {original_expr}\n")

print("Column mapping:")
for old, new in column_mapping.items():
    print(f"  {old} → {new}")
print()

transformed_expr = fql.replace_column_names(original_expr, column_mapping)
print("Transformed expression:")
print(f"  {transformed_expr}\n")

# Verify the transformation
original_cols = fql.extract_columns(original_expr)
transformed_cols = fql.extract_columns(transformed_expr)

print("Verification:")
print(f"  Original columns: {original_cols}")
print(f"  Transformed columns: {transformed_cols}")

### 3.2 Complex Expression Transformation

In [ ]:
# Transform custom metric with aggregations
custom_metric_expr = 'sum(if(fp(), "transaction_value", 0)) / count(if(fp(), 1, 0))'

metric_mapping = {"transaction_value": "txn_amount"}

print("Complex Expression Transformation:\n")
print("Original custom metric:")
print(f"  {custom_metric_expr}\n")

transformed_metric = fql.replace_column_names(custom_metric_expr, metric_mapping)
print("Transformed custom metric:")
print(f"  {transformed_metric}\n")

# Verify functions are preserved
original_functions = fql.get_fql_functions(custom_metric_expr)
transformed_functions = fql.get_fql_functions(transformed_metric)

print("Verification:")
print(f"  Functions preserved: {original_functions == transformed_functions}")
print(f"  Functions: {transformed_functions}")

### 3.3 Interactive Column Mapping Builder

In [ ]:
# Helper function to build column mapping between two models
def build_column_mapping_interactive(
    source_columns: Set[str], target_columns: Set[str]
) -> Dict[str, str]:
    """Interactively build a column mapping between source and target schemas.

    This is a simplified version for demonstration. In practice, you might:
    - Use fuzzy matching to suggest mappings
    - Allow user input for manual mapping
    - Handle partial mappings
    """
    mapping = {}

    # Exact matches (case-insensitive)
    target_lower = {col.lower(): col for col in target_columns}

    for source_col in source_columns:
        if source_col.lower() in target_lower:
            target_col = target_lower[source_col.lower()]
            if source_col != target_col:
                mapping[source_col] = target_col

    return mapping


# Example: Two models with similar but different schemas
source_schema = {"age", "geography", "credit_score", "income", "employment_status"}
target_schema = {
    "customer_age",
    "location",
    "fico_score",
    "annual_income",
    "employment_status",
}

print("Interactive Column Mapping Builder:\n")
print(f"Source schema: {sorted(source_schema)}")
print(f"Target schema: {sorted(target_schema)}\n")

# Auto-detect exact matches
auto_mapping = build_column_mapping_interactive(source_schema, target_schema)
print(f"Auto-detected mappings: {auto_mapping}\n")

# Identify unmapped columns
unmapped_source = source_schema - set(auto_mapping.keys()) - target_schema
unmapped_target = target_schema - set(auto_mapping.values()) - source_schema

print(f"Unmapped source columns: {unmapped_source}")
print(f"Unmapped target columns: {unmapped_target}\n")

# Manual mapping for demonstration
print("Suggested manual mappings:")
manual_mapping = {
    "age": "customer_age",
    "geography": "location",
    "credit_score": "fico_score",
    "income": "annual_income",
}

for source, target in manual_mapping.items():
    print(f"  {source} → {target}")

# Combined mapping
full_mapping = {**auto_mapping, **manual_mapping}
print(f"\nFinal mapping: {full_mapping}")

### 3.4 End-to-End Migration Example

In [ ]:
# Complete workflow: Migrate a segment from one model to another
print("End-to-End Segment Migration Workflow:\n")

# Source segment
source_segment_name = "High Risk Customers"
source_segment_expr = (
    '"age" < 25 and "credit_score" < 650 and "geography" == \'California\''
)

print("Step 1: Source Segment")
print(f"  Name: {source_segment_name}")
print(f"  Expression: {source_segment_expr}\n")

# Extract columns from source expression
print("Step 2: Extract Column References")
source_cols = fql.extract_columns(source_segment_expr)
print(f"  Columns: {source_cols}\n")

# Define target schema and mapping
target_schema_cols = {"customer_age", "fico_score", "location", "income_bracket"}
migration_mapping = {
    "age": "customer_age",
    "credit_score": "fico_score",
    "geography": "location",
}

print("Step 3: Apply Column Mapping")
print(f"  Target schema: {target_schema_cols}")
print(f"  Mapping: {migration_mapping}\n")

# Transform expression
target_segment_expr = fql.replace_column_names(source_segment_expr, migration_mapping)
print("Step 4: Transform Expression")
print(f"  Transformed: {target_segment_expr}\n")

# Validate against target schema
print("Step 5: Validate Against Target Schema")
is_valid, missing_cols = fql.validate_column_references(
    target_segment_expr, target_schema_cols
)

if is_valid:
    print("  ✅ Expression is valid for target model")
    print("\nStep 6: Ready to Create Segment")
    if DRY_RUN:
        print("  [DRY RUN] Would create segment:")
        print(f"    Name: {source_segment_name}")
        print(f"    Expression: {target_segment_expr}")
    else:
        print("  Set DRY_RUN=False to create the segment")
else:
    print(f"  ✗ Validation failed - missing columns: {missing_cols}")
    print("  Cannot proceed with migration")

### 3.5 Batch Migration with Validation

In [ ]:
# Migrate multiple segments at once
source_segments = [
    {"name": "High Risk", "expr": '"age" < 25 and "credit_score" < 650'},
    {"name": "Premium Customers", "expr": '"income" > 100000 and "credit_score" > 750'},
    {"name": "California Only", "expr": "\"geography\" == 'California'"},
]

batch_mapping = {
    "age": "customer_age",
    "credit_score": "fico_score",
    "income": "annual_income",
    "geography": "location",
}

target_cols = {"customer_age", "fico_score", "annual_income", "location"}

print("Batch Segment Migration:\n")

results = []
for segment in source_segments:
    print(f"Processing: {segment['name']}")

    # Transform
    transformed = fql.replace_column_names(segment["expr"], batch_mapping)

    # Validate
    is_valid, missing = fql.validate_column_references(transformed, target_cols)

    result = {
        "name": segment["name"],
        "original": segment["expr"],
        "transformed": transformed,
        "valid": is_valid,
        "missing": missing,
    }
    results.append(result)

    status = "✓" if is_valid else "✗"
    print(f"  {status} Transformed: {transformed}")
    if not is_valid:
        print(f"    Missing: {missing}")
    print()

# Summary
valid_count = sum(1 for r in results if r["valid"])
print("Summary:")
print(f"  Total segments: {len(results)}")
print(f"  Valid after transformation: {valid_count}")
print(f"  Failed validation: {len(results) - valid_count}")

if valid_count == len(results):
    print("\n✅ All segments ready for migration!")
else:
    print(f"\n⚠️ {len(results) - valid_count} segment(s) need manual review")

---

## Section 4: Advanced Patterns and Best Practices

Combining multiple utilities for robust FQL workflows.

---

## Section 4: Advanced Patterns and Best Practices

Combining multiple utilities for robust FQL workflows.

### 4.1 Comprehensive Expression Validation Pipeline

In [ ]:
def validate_fql_comprehensive(expression: str, valid_columns: Set[str]) -> Dict:
    """Run comprehensive validation on an FQL expression.

    Returns:
        Dictionary with validation results and metadata
    """
    results = {
        "expression": expression,
        "checks": {},
        "all_valid": True,
        "warnings": [],
        "metadata": {},
    }

    # Check 1: Syntax validation
    is_valid, error = fql.validate_fql_syntax(expression)
    results["checks"]["syntax"] = {"valid": is_valid, "error": error}
    if not is_valid:
        results["all_valid"] = False

    # Check 2: Column references
    is_valid, missing = fql.validate_column_references(expression, valid_columns)
    results["checks"]["columns"] = {"valid": is_valid, "missing": missing}
    if not is_valid:
        results["all_valid"] = False

    # Metadata: Extract columns
    columns = fql.extract_columns(expression)
    results["metadata"]["columns"] = list(columns)
    results["metadata"]["column_count"] = len(columns)

    # Metadata: Extract functions
    functions = fql.get_fql_functions(expression)
    results["metadata"]["functions"] = list(functions)
    results["metadata"]["has_aggregations"] = not fql.is_simple_filter(expression)

    # Warning: Check for complexity
    if len(columns) > 5:
        results["warnings"].append(f"Complex expression with {len(columns)} columns")

    if len(functions) > 3:
        results["warnings"].append(f"Multiple nested functions ({len(functions)})")

    return results


# Test the pipeline
test_schema = {"age", "income", "credit_score", "status", "region"}

test_expressions = [
    '"age" > 30 and "status" == \'active\'',
    'sum(if(fp(), "transaction_value", 0))',  # Missing column
    '"age" > 30 and "income" > 50000 and "credit_score" >= 700',
]

print("Comprehensive Validation Pipeline:\n")

for expr in test_expressions:
    print(f"Expression: {expr}")
    results = validate_fql_comprehensive(expr, test_schema)

    # Show results
    overall = "✅ PASS" if results["all_valid"] else "✗ FAIL"
    print(f"  Overall: {overall}")

    # Checks
    for check_name, check_result in results["checks"].items():
        status = "✓" if check_result["valid"] else "✗"
        print(
            f"  {status} {check_name.title()}: {'Valid' if check_result['valid'] else check_result.get('error') or check_result.get('missing')}"
        )

    # Metadata
    print("  Metadata:")
    print(f"    Columns: {results['metadata']['columns']}")
    print(f"    Functions: {results['metadata']['functions']}")
    print(f"    Has aggregations: {results['metadata']['has_aggregations']}")

    # Warnings
    if results["warnings"]:
        print("  Warnings:")
        for warning in results["warnings"]:
            print(f"    ⚠️ {warning}")

    print()

### 4.2 Expression Comparison and Deduplication

In [ ]:
# Use normalization to find duplicate expressions
segment_expressions = [
    '"age" > 30 and "status" == \'active\'',
    '"age"   >   30   and   "status"   ==   \'active\'',  # Same, different whitespace
    '"status" == \'active\' and "age" > 30',  # Different order, semantically same
    '"age" > 25 and "status" == \'active\'',  # Actually different
]

print("Expression Comparison and Deduplication:\n")

normalized_map = {}
for i, expr in enumerate(segment_expressions, 1):
    normalized = fql.normalize_expression(expr)

    if normalized in normalized_map:
        print(f"Expression {i}: DUPLICATE of Expression {normalized_map[normalized]}")
    else:
        print(f"Expression {i}: UNIQUE")
        normalized_map[normalized] = i

    print(f"  Original:    {expr}")
    print(f"  Normalized:  {normalized}")
    print()

print(
    f"Summary: {len(normalized_map)} unique expressions out of {len(segment_expressions)} total"
)

### 4.3 Safe Expression Modification Workflow

In [ ]:
def safe_column_replacement(
    expression: str,
    mapping: Dict[str, str],
    target_schema: Set[str],
    dry_run: bool = True,
) -> Dict:
    """Safely replace column names with validation.

    Args:
        expression: Original FQL expression
        mapping: Column name mapping (old -> new)
        target_schema: Valid columns in target schema
        dry_run: If True, only simulate the change

    Returns:
        Dictionary with transformation results
    """
    result = {
        "original": expression,
        "transformed": None,
        "success": False,
        "errors": [],
        "warnings": [],
        "dry_run": dry_run,
    }

    # Step 1: Validate original syntax
    is_valid, error = fql.validate_fql_syntax(expression)
    if not is_valid:
        result["errors"].append(f"Original expression has syntax error: {error}")
        return result

    # Step 2: Extract columns from original
    original_cols = fql.extract_columns(expression)

    # Step 3: Check if all columns to be replaced exist
    cols_to_replace = set(mapping.keys())
    missing_in_expr = cols_to_replace - original_cols
    if missing_in_expr:
        result["warnings"].append(
            f"Columns in mapping not found in expression: {missing_in_expr}"
        )

    # Step 4: Apply transformation
    transformed = fql.replace_column_names(expression, mapping)
    result["transformed"] = transformed

    # Step 5: Validate transformed syntax
    is_valid, error = fql.validate_fql_syntax(transformed)
    if not is_valid:
        result["errors"].append(f"Transformed expression has syntax error: {error}")
        return result

    # Step 6: Validate against target schema
    is_valid, missing = fql.validate_column_references(transformed, target_schema)
    if not is_valid:
        result["errors"].append(
            f"Transformed expression references missing columns: {missing}"
        )
        return result

    # Step 7: Success!
    result["success"] = True

    return result


# Test safe replacement
print("Safe Expression Modification Workflow:\n")

test_expr = '"age" > 30 and "credit_score" >= 700'
test_mapping = {"age": "customer_age", "credit_score": "fico_score"}
test_target_schema = {"customer_age", "fico_score", "location", "income"}

result = safe_column_replacement(
    test_expr, test_mapping, test_target_schema, dry_run=DRY_RUN
)

print(f"Original: {result['original']}")
print(f"Transformed: {result['transformed']}")
print(f"Status: {'✅ SUCCESS' if result['success'] else '✗ FAILED'}")

if result["errors"]:
    print("\nErrors:")
    for error in result["errors"]:
        print(f"  ✗ {error}")

if result["warnings"]:
    print("\nWarnings:")
    for warning in result["warnings"]:
        print(f"  ⚠️ {warning}")

if result["dry_run"] and result["success"]:
    print("\nℹ️ DRY RUN MODE - No changes made")
    print("   Set DRY_RUN=False to apply transformation")

### 4.4 Expression Analysis Report

In [ ]:
def analyze_expression_complexity(expression: str) -> Dict:
    """Analyze FQL expression complexity and characteristics."""
    return {
        "length": len(expression),
        "columns": fql.extract_columns(expression),
        "column_count": len(fql.extract_columns(expression)),
        "functions": fql.get_fql_functions(expression),
        "function_count": len(fql.get_fql_functions(expression)),
        "is_simple_filter": fql.is_simple_filter(expression),
        "has_aggregations": not fql.is_simple_filter(expression),
        "and_conditions": len(fql.split_fql_and_condition(expression)),
        "complexity_score": (
            len(fql.extract_columns(expression)) * 1.0
            + len(fql.get_fql_functions(expression)) * 2.0
            + len(fql.split_fql_and_condition(expression)) * 0.5
        ),
    }


# Analyze various expressions
expressions_to_analyze = [
    '"age" > 30',
    '"age" > 30 and "status" == \'active\'',
    "\"age\" > 30 and \"status\" == 'active' and \"region\" in ['West', 'East']",
    "sum(if(fp(), 1, 0))",
    'sum(if(fp(), "value", 0)) / count(if(tp(), 1, 0))',
]

print("Expression Complexity Analysis:\n")

for expr in expressions_to_analyze:
    analysis = analyze_expression_complexity(expr)

    print(f"Expression: {expr}")
    print(
        f"  Type: {'Simple Filter' if analysis['is_simple_filter'] else 'Aggregation/Metric'}"
    )
    print(
        f"  Columns: {analysis['column_count']} ({', '.join(analysis['columns']) if analysis['columns'] else 'none'})"
    )
    print(
        f"  Functions: {analysis['function_count']} ({', '.join(analysis['functions']) if analysis['functions'] else 'none'})"
    )
    print(f"  AND conditions: {analysis['and_conditions']}")
    print(f"  Complexity score: {analysis['complexity_score']:.1f}")
    print()

## ⚠️ Critical Limitations & What These Utilities Don't Solve

**While these utilities are powerful, it's important to understand their limitations:**

### What is Solved ✅

1. **Cross-Model Migration**
   - Column name mapping and transformation
   - Schema validation before migration
   - Batch migration with error handling

2. **Reference Management**
   - Find all Charts/Alerts using a metric
   - Safe metric updates with automatic reference migration
   - Prevents broken dashboards

3. **FQL Testing**
   - Local syntax validation (fast pre-check)
   - Real testing via temporary metrics
   - Automatic cleanup

### What is Not Solved ❌

1. **Core API Limitation: Metric Immutability**
   - Custom Metrics still cannot be truly edited
   - `safe_update_metric()` is a delete+recreate workaround
   - The Fiddler API itself does not support metric modification

2. **Dry-Run Limitations**
   - Local validation **cannot** test Fiddler-specific functions:
     - `tp()`, `fp()`, `tn()`, `fn()` - require prediction data
     - `jsd()`, `psi()` - require baseline data in Fiddler
     - Data integrity functions - require model spec
   - Testing **must** create temporary metrics in Fiddler
   - No true "preview" without API call

3. **Reference Migration Caveats**
   - Chart updates use unofficial API (may change)
   - Alert recreation may reset notification configs
   - Cannot rollback UUID changes after commit
   - Some alert properties may not be preserved

4. **Semantic Validation Gaps**
   - Cannot detect division by zero
   - Cannot validate data type compatibility
   - Cannot check runtime performance
   - Cannot estimate calculation cost

---

## Summary

### Key Takeaways

1. **FQL Syntax Fundamentals**
   * Column names in double quotes: `"column_name"`
   * String values in single quotes: `'value'`
   * Numeric values unquoted: `42`

2. **Core Utility Functions**
   * `extract_columns()` - Find all column references
   * `validate_fql_syntax()` - Catch syntax errors
   * `validate_column_references()` - Check schema compatibility
   * `replace_column_names()` - Transform expressions for migration
   * `normalize_expression()` - Standardize formatting
   * `get_fql_functions()` - Identify functions used
   * `is_simple_filter()` - Distinguish filters from aggregations
   * `split_fql_and_condition()` - Break down complex conditions

3. **Common Use Cases**
   * **Asset migration:** Copy segments/metrics between models with different schemas
   * **Validation:** Verify expressions before deployment
   * **Analysis:** Understand expression complexity and dependencies
   * **Deduplication:** Find semantically identical expressions

4. **Best Practices**
   * Always validate syntax before applying transformations
   * Check schema compatibility after column name replacements
   * Use dry-run mode for testing transformations
   * Normalize expressions when comparing for equality
   * Build comprehensive validation pipelines for production workflows

### Common Gotchas

* **Quote consistency:** Mixing single/double quotes will cause syntax errors
* **Partial column name matches:** Use word boundaries in replacements to avoid partial matches
* **Expression order:** `"a" and "b"` vs `"b" and "a"` are semantically same but string-different
* **Function detection:** `split_fql_and_condition()` uses simple pattern matching and may not handle complex nested cases
* **Schema validation:** Only checks if columns exist, not data types or value compatibility

### When to Use Each Function

| Function | Use When |
|----------|----------|
| `extract_columns()` | You need to know what data columns an expression depends on |
| `validate_fql_syntax()` | Before saving expressions to catch obvious syntax errors |
| `validate_column_references()` | Migrating assets or checking if expression will work on a model |
| `replace_column_names()` | Copying assets between models with different column names |
| `normalize_expression()` | Comparing expressions or finding duplicates |
| `get_fql_functions()` | Analyzing expression complexity or checking for specific functions |
| `is_simple_filter()` | Determining if an expression can be used as a segment filter |
| `split_fql_and_condition()` | Breaking down complex filters into individual conditions |

### Resources

* **Fiddler FQL Documentation:** https://docs.fiddler.ai
* **fiddler_utils package:** See `fiddler_utils/fql.py` for source code
* **Additional utilities:** See `/misc-utils/README.md` for other helpful tools

---

## Section 5: UUID Reference Management & Safe Metric Updates

⚠️ Custom Metrics cannot be modified once created. When you delete and recreate a metric (to "update" it), it gets a **new UUID**, which breaks all Charts and Alerts that reference the old UUID.

This section demonstrates:
1. The UUID breakage problem
2. How to find all references before updating
3. Safe update workflow with automatic reference migration

### 5.1 Import Reference Management Utilities

In [ ]:
# Import reference management utilities
from fiddler_utils.assets.references import (
    find_all_metric_references,
    safe_update_metric,
)

print("✓ Reference management utilities imported")

### 5.2 Demonstrating the UUID Problem

In [ ]:
# Demonstrate the UUID breakage problem
# NOTE: This is a demonstration - run only if you understand the impact!

if URL and TOKEN and model and not DRY_RUN:
    print("⚠️ WARNING: This demonstration will:")
    print("  1. Create a test custom metric")
    print("  2. Delete and recreate it (simulating an update)")
    print("  3. Show that the UUID changes")
    print()
    print("Set DRY_RUN=True to skip this demonstration")
    print()
    
    # Create a test metric
    test_metric = fdl.CustomMetric(
        model_id=model.id,
        name='__demo_uuid_problem',
        description='TEST METRIC - Demonstrating UUID change on recreation',
        definition='sum(if(fp(), 1, 0))'
    )
    test_metric.create()
    
    original_id = test_metric.id
    print("✓ Created metric '__demo_uuid_problem'")
    print(f"  Original UUID: {original_id}")
    print()
    
    # Delete and recreate (simulating an update)
    test_metric.delete()
    print("Deleted metric...")
    print()
    
    # Recreate with same name
    test_metric_v2 = fdl.CustomMetric(
        model_id=model.id,
        name='__demo_uuid_problem',
        description='TEST METRIC - Recreated version',
        definition='sum(if(fn(), 1, 0))'  # Changed definition
    )
    test_metric_v2.create()
    
    new_id = test_metric_v2.id
    print(f"✓ Recreated metric with same name")
    print(f"  New UUID: {new_id}")
    print()
    
    # Compare UUIDs
    print("🔴 PROBLEM DEMONSTRATED:")
    print(f"  UUIDs are different: {original_id != new_id}")
    print(f"  Any Charts/Alerts referencing {original_id} are now BROKEN")
    print()
    
    # Cleanup
    test_metric_v2.delete()
    print("✓ Cleaned up test metric")
else:
    print("DRY_RUN=True - Skipping UUID problem demonstration")
    print()
    print("The Problem:")
    print("  1. You create a metric → Gets UUID abc-123")
    print("  2. Charts and Alerts reference UUID abc-123")
    print("  3. You delete and recreate metric → Gets NEW UUID xyz-789")
    print("  4. Charts and Alerts still reference abc-123 (BROKEN!)")

### 5.3 Finding All References Before Updating

In [ ]:
# Find all references to a custom metric before updating it
if URL and TOKEN and model:
    # Get a custom metric (use first available)
    custom_metrics = list(fdl.CustomMetric.list(model_id=model.id))
    
    if custom_metrics:
        # Use first metric for demonstration
        metric = custom_metrics[0]
        
        print(f"Analyzing references to metric: {metric.name}")
        print(f"  Metric ID: {metric.id}")
        print(f"  Definition: {metric.definition}")
        print()
        
        # Find all references
        try:
            refs = find_all_metric_references(
                metric_id=metric.id,
                project_id=project.id,
                url=URL,
                token=TOKEN
            )
            
            print("Reference Discovery Results:")
            print(f"  Charts: {refs['chart_count']}")
            print(f"  Alerts: {refs['alert_count']}")
            print(f"  Total References: {refs['total_count']}")
            print()
            
            if refs['has_references']:
                print("⚠️ WARNING: Deleting this metric will break:")
                
                if refs['charts']:
                    print(f"\n  Charts ({len(refs['charts'])}):")
                    for chart in refs['charts'][:5]:  # Show first 5
                        print(f"    - {chart.get('title', 'Untitled')}")
                    if len(refs['charts']) > 5:
                        print(f"    ... and {len(refs['charts']) - 5} more")
                
                if refs['alerts']:
                    print(f"\n  Alerts ({len(refs['alerts'])}):")
                    for alert in refs['alerts'][:5]:  # Show first 5
                        print(f"    - {alert.name}")
                    if len(refs['alerts']) > 5:
                        print(f"    ... and {len(refs['alerts']) - 5} more")
            else:
                print("✓ No references found - safe to delete/update")
                
        except Exception as e:
            print(f"Error finding references: {e}")
            print("Note: Chart API may require url/token parameters")
    else:
        print("No custom metrics found in this model")
else:
    print("⚠️ Please set URL, TOKEN, and ensure model is loaded")

### 5.4 Safe Metric Update with Automatic Reference Migration

In [ ]:
# Safely update a metric with automatic reference migration
if URL and TOKEN and model and not DRY_RUN:
    print("⚠️ This will update a real metric and migrate references")
    print("Set DRY_RUN=True to see the workflow without making changes")
    print()
    
    # For demonstration, create a test metric
    print("Step 1: Creating test metric...")
    test_metric = fdl.CustomMetric(
        model_id=model.id,
        name='__demo_safe_update',
        description='Test metric for safe update demonstration',
        definition='sum(if(fp(), 1, 0))'
    )
    test_metric.create()
    print(f"  ✓ Created metric (ID: {test_metric.id})")
    print()
    
    # Use safe update
    print("Step 2: Updating metric with safe_update_metric()...")
    new_metric, report = safe_update_metric(
        metric=test_metric,
        new_definition='sum(if(fn(), 1, 0))',  # Changed to false negatives
        auto_migrate=True,
        project_id=project.id,
        url=URL,
        token=TOKEN
    )
    
    print()
    print("Update Complete!")
    print(f"  Old ID: {report['old_metric_id']}")
    print(f"  New ID: {report['new_metric_id']}")
    print(f"  Metric Name: {report['metric_name']}")
    print()
    print("Migration Results:")
    print(f"  Migrated: {report['migrated_count']} references")
    print(f"  Failed: {report['failed_count']} references")
    print()
    
    if report['chart_migrations']:
        print("Chart Migrations:")
        for migration in report['chart_migrations']:
            status = "✓" if migration['success'] else "✗"
            print(f"  {status} {migration['title']}")
    
    if report['alert_migrations']:
        print("Alert Migrations:")
        for migration in report['alert_migrations']:
            status = "✓" if migration['success'] else "✗"
            print(f"  {status} {migration['name']}")
    
    # Cleanup
    print()
    print("Cleaning up test metric...")
    new_metric.delete()
    print("✓ Done")
    
else:
    print("DRY_RUN Mode - Safe Update Workflow:")
    print()
    print("The safe_update_metric() function:")
    print("  1. Finds all Charts and Alerts referencing the metric")
    print("  2. Validates the new FQL definition")
    print("  3. Deletes the old metric")
    print("  4. Creates new metric with same name (gets new UUID)")
    print("  5. Updates ALL Charts to reference the new UUID")
    print("  6. Updates ALL Alerts to reference the new UUID")
    print("  7. Returns migration report")
    print()
    print("Result: Metric is updated WITHOUT breaking dashboards!")
    print()
    print("Example usage:")
    print("""
    new_metric, report = safe_update_metric(
        metric=my_metric,
        new_definition='sum(if(tp(), 1, 0))',
        auto_migrate=True,
        project_id=project.id,
        url=URL,
        token=TOKEN
    )
    
    print(f"Migrated {report['migrated_count']} references")
    """)

---

## Section 6: Testing FQL Before Creating Metrics

**The Challenge:** Fiddler does not provide a "validate" or "dry-run" API for FQL. The ONLY way to know if your metric definition will work is to create it.

**The Solution:** Use temporary metrics for testing:
1. Local pre-validation (fast, catches syntax errors)
2. Temporary metric testing in Fiddler (real validation)
3. Automatic cleanup

This approach lets you iterate on FQL definitions without polluting your metrics list.

### 6.1 Import Testing Utilities

In [ ]:
# Import FQL testing utilities
from fiddler_utils.testing import (
    batch_test_metrics,
    cleanup_orphaned_test_metrics,
    test_metric_definition,
    validate_and_preview_metric,
    validate_metric_syntax_local,
)

print("✓ FQL testing utilities imported")

### 6.2 Local Pre-Validation (Fast)

Local validation catches obvious errors without making API calls. It's fast but limited.

In [ ]:
# Fast local validation
if model:
    # Test various FQL expressions locally
    test_expressions = [
        ('sum(if(fp(), 1, 0))', "Valid aggregation"),
        ('sum(if(fp(), 1, 0)', "Missing closing paren"),
        ('"age" > 30', "Simple filter - not valid for custom metrics"),
        ('sum(if(fp(), "nonexistent_column", 0))', "References missing column"),
    ]
    
    print("Local Pre-Validation Results:\n")
    
    for expr, description in test_expressions:
        print(f"Testing: {description}")
        print(f"  Expression: {expr}")
        
        result = validate_metric_syntax_local(expr, model)
        
        if result['valid']:
            print("  ✓ Passed local validation")
        else:
            print(f"  ✗ Failed: {result['errors']}")
        
        if result['has_warnings']:
            print(f"  ⚠️  Warnings: {result['warnings']}")
        
        print()
    
    print("Note: Local validation only catches syntax and schema errors.")
    print("It cannot validate Fiddler-specific functions (tp(), fp(), jsd(), etc.)")
else:
    print("⚠️ Model not loaded - skipping local validation demo")

### 6.3 Real Testing with Temporary Metrics

The only way to truly validate FQL is to create a metric in Fiddler. We use temporary metrics with auto-cleanup.

In [ ]:
# Test FQL by creating temporary metric in Fiddler
if URL and TOKEN and model and not DRY_RUN:
    print("Testing FQL with Temporary Metrics\n")
    
    # Test 1: Valid metric
    print("Test 1: Valid FQL expression")
    result = test_metric_definition(
        model_id=model.id,
        definition='sum(if(fp(), 1, 0))',
        cleanup=True
    )
    
    if result['valid']:
        print("  ✓ Expression is valid!")
        print(f"    Temp metric created: {result['temp_metric_name']}")
        print(f"    Cleaned up: {result['cleaned_up']}")
    else:
        print(f"  ✗ Expression failed: {result['error']}")
    print()
    
    # Test 2: Invalid metric (syntax error)
    print("Test 2: Invalid FQL (syntax error)")
    result = test_metric_definition(
        model_id=model.id,
        definition='sum(if(fp(), 1, 0',  # Missing closing paren
        cleanup=True
    )
    
    if result['valid']:
        print("  ✓ Expression is valid!")
    else:
        print(f"  ✗ Expression failed (expected): {result['error']}")
    print()
    
    # Test 3: Invalid metric (Fiddler-specific error)
    print("Test 3: FQL that might fail Fiddler validation")
    result = test_metric_definition(
        model_id=model.id,
        definition='sum(fp()) / 0',  # Division by zero
        cleanup=True
    )
    
    if result['valid']:
        print("  ✓ Expression passed Fiddler validation")
        print("    (Note: Fiddler may still accept this)")
    else:
        print(f"  ✗ Expression failed: {result['error']}")
    
else:
    print("DRY_RUN Mode - Testing Workflow Explanation:\n")
    print("test_metric_definition() workflow:")
    print("  1. Generate unique temp metric name (__test_<timestamp>_<random>)")
    print("  2. Create metric in Fiddler")
    print("  3. Fiddler validates the FQL definition")
    print("  4. If successful, return success")
    print("  5. If failed, capture error message")
    print("  6. Delete temp metric (cleanup)")
    print()
    print("This is the ONLY way to truly validate FQL!")
    print("Local validation cannot test tp(), fp(), jsd(), etc.")

### 6.4 Complete Validation Workflow

Combine local and Fiddler testing for comprehensive validation.

In [ ]:
# Complete validation workflow
if URL and TOKEN and model and not DRY_RUN:
    print("Complete Validation Workflow\n")
    
    # Test a metric definition with both local and Fiddler validation
    definition = 'sum(if(fp(), 1, 0)) / sum(1)'
    
    print(f"Testing definition: {definition}")
    print()
    
    result = validate_and_preview_metric(
        model_id=model.id,
        definition=definition
    )
    
    # Show local validation results
    if result['local_validation']:
        print("Step 1: Local Validation")
        local = result['local_validation']
        if local['valid']:
            print("  ✓ Passed")
        else:
            print(f"  ✗ Failed: {local['errors']}")
        
        if local['has_warnings']:
            print(f"  ⚠️  Warnings:")
            for warning in local['warnings']:
                print(f"    - {warning}")
        print()
    
    # Show Fiddler test results
    if result['fiddler_test']:
        print("Step 2: Fiddler Validation")
        fiddler = result['fiddler_test']
        if fiddler['valid']:
            print("  ✓ Passed")
        else:
            print(f"  ✗ Failed: {fiddler['error']}")
        print()
    
    # Show recommendation
    print("Recommendation:")
    print(f"  {result['recommendation']}")
    print()
    
    if result['valid']:
        print("You can now safely create the real metric!")
        print("""
        metric = fdl.CustomMetric(
            model_id=model.id,
            name='fp_rate',
            definition='{}',
        )
        metric.create()
        """.format(definition))
    
else:
    print("DRY_RUN Mode - Complete Workflow:\n")
    print("validate_and_preview_metric() combines:")
    print("  1. Fast local validation (syntax, schema)")
    print("  2. Real Fiddler testing (creates temp metric)")
    print("  3. Automatic cleanup")
    print("  4. Clear recommendation")
    print()
    print("Use this before creating any custom metric!")

### 6.5 Batch Testing Multiple Metrics

In [ ]:
# Test multiple metric definitions at once
if URL and TOKEN and model and not DRY_RUN:
    print("Batch Testing Multiple Metrics\n")
    
    # Define multiple metrics to test
    metric_definitions = [
        {'name': 'FP Count', 'definition': 'sum(if(fp(), 1, 0))'},
        {'name': 'FN Count', 'definition': 'sum(if(fn(), 1, 0))'},
        {'name': 'TP Count', 'definition': 'sum(if(tp(), 1, 0))'},
        {'name': 'Accuracy', 'definition': 'sum(if(tp() or tn(), 1, 0)) / sum(1)'},
    ]
    
    # Batch test
    results = batch_test_metrics(
        model_id=model.id,
        definitions=metric_definitions,
        delay_between_tests=0.5
    )
    
    # Show results
    print("Batch Test Results:\n")
    valid_count = sum(1 for r in results if r['valid'])
    
    for result in results:
        status = "✓" if result['valid'] else "✗"
        error_msg = f" - {result['error']}" if not result['valid'] else ""
        print(f"{status} {result['name']}{error_msg}")
    
    print()
    print(f"Summary: {valid_count}/{len(results)} definitions are valid")
    print()
    
    if valid_count == len(results):
        print("✅ All definitions validated! Ready to create metrics.")
    else:
        print("⚠️  Some definitions failed. Review errors above.")
    
else:
    print("DRY_RUN Mode - Batch Testing:\n")
    print("batch_test_metrics() efficiently tests multiple definitions:")
    print("  - Creates/deletes temp metric for each")
    print("  - Adds delay between tests (avoid rate limiting)")
    print("  - Returns results for all definitions")
    print()
    print("Use this when developing a suite of custom metrics!")

### 6.6 Cleanup Orphaned Test Metrics

If testing was interrupted, temporary metrics may remain. Clean them up.

In [ ]:
# Cleanup any orphaned test metrics
if URL and TOKEN and model and not DRY_RUN:
    print("Cleaning Up Orphaned Test Metrics\n")
    
    deleted = cleanup_orphaned_test_metrics(model.id)
    
    if deleted > 0:
        print(f"✓ Cleaned up {deleted} orphaned test metrics")
    else:
        print("✓ No orphaned test metrics found")
    
else:
    print("DRY_RUN Mode - Cleanup:\n")
    print("cleanup_orphaned_test_metrics() finds and deletes:")
    print("  - All metrics starting with '__test_'")
    print("  - Leftover from interrupted testing")
    print()
    print("Run this periodically to keep your metrics clean!")